# Using API to Generate the transcription in English

In [ ]:
import os
from dotenv import load_dotenv
import requests
import io
from pydub import AudioSegment

# Load environment variables from .env file
load_dotenv()
# Access your API key
SARVAM_API_KEY = os.getenv("SHARVAM_API_KEY")

# Helper Functions

In [ ]:
def translate_audio(audio, SARVAM_API_KEY):

    # API endpoint for speech-to-text translation
    api_url = "https://api.sarvam.ai/speech-to-text-translate"

    # Headers containing the API subscription key
    headers = {
        "api-subscription-key": SARVAM_API_KEY  # Replace with your API key
    }

    # Data payload for the translation request
    model_data = {
        "model": "saaras:v2",  # Specify the model to be used
        "with_diarization": False  # Set to True for speaker diarization
    }


    chunk_buffer = io.BytesIO()
    audio.export(chunk_buffer, format="wav")
    chunk_buffer.seek(0)  # Reset the pointer to the start of the stream

    # Prepare the file for the API request
    files = {'file': ('audiofile.wav', chunk_buffer, 'audio/wav')}

    try:
        # Make the POST request to the API
        response = requests.post(api_url, headers=headers, files=files, data=model_data)
        if response.status_code == 200 or response.status_code == 201:
            response_data = response.json()
            transcript = response_data.get("transcript", "")
        else:
            # Handle failed requests
            print(f"failed with status code: {response.status_code}")
            print("Response:", response.text)
    except Exception as e:
        # Handle any exceptions during the request
        print(f"Error processing chunk {e}")
    finally:
        # Ensure the buffer is closed after processing
        chunk_buffer.close()

    return transcript
